In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
from copy import deepcopy
from pathlib import Path


import pandas as pd
import numpy as np
import torch
import torchio as tio
import h5py
from ipywidgets import interact
import matplotlib.pyplot as plt


dir2 = os.path.abspath('..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)

In [2]:
from research.data.kamitani_2019 import Kamitani2019, RawKamitani2019, Kamitani2019H5

# Load the dataset with fMRIprep applied

root = "X:\\Datasets\\Deep-Image-Reconstruction\\"
raw_dataset = RawKamitani2019(root)

C:\Users\Cefir\anaconda3\envs\Neurophysiological-Data-Decoding\lib\site-packages\bids\layout\models.py:148: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  warnings.warn("The 'extension' entity currently excludes the leading dot ('.'). "


In [9]:
def require_dataset(group, name, data):
    group.require_dataset(name, shape=data.shape, dtype=data.dtype)
    group[name][:] = data

In [11]:
# Initial creation

break

with h5py.File(Path(root) / "derivatives" / "kamitani2019.hdf5", "w") as f:
    for subject in raw_dataset.subjects:
        print(subject['name'])
        
        f.create_group(f'{subject["name"]}/anatomy/')
        
        anat_image_names = ['T1w', 'brain_mask', 'csf_mask', 'gm_mask', 'wm_mask', 
                            'LH_FFA', 'LH_hV4', 'LH_HVC', 'LH_LOC', 'LH_PPA', 'LH_V1d', 
                            'LH_V1v', 'LH_V2d', 'LH_V2v', 'LH_V3d', 'LH_V3v', 
                            'RH_FFA', 'RH_hV4', 'RH_HVC', 'RH_LOC', 'RH_PPA', 'RH_V1d', 
                            'RH_V1v', 'RH_V2d', 'RH_V2v', 'RH_V3d', 'RH_V3v']
        for anat_image_name in anat_image_names:
            anat_image = deepcopy(subject[anat_image_name])
            anat_image.load()
            anat_dset = f.create_dataset(f'{subject["name"]}/anatomy/{anat_image_name}', data=anat_image['data'])
            for k, v in anat_image.items():
                if k == 'data':
                    continue
                anat_dset.attrs[k] = v
        
        for session_type, sessions in subject['func_sessions'].items():
            print(session_type)
            for i, run_image in enumerate(sessions):
                run_image = deepcopy(run_image)
                run_image.load()
                run = f.create_group(f'{subject["name"]}/{session_type}/runs/run_{i:03}/')
                run_dataset = run.create_dataset('data', data=run_image['data'])
                
                print(run)
                data = run_dataset[:]
                
                for k, v in run_image.items():
                    if k == 'data':
                        continue
                    if isinstance(v, pd.DataFrame):
                        v = v.to_csv()
                    run.attrs[k] = v

                H, W, D, T = data.shape
                
                data = torch.from_numpy(data).cuda()
                
                voxel_mean = data.mean(dim=3).cpu().numpy()
                voxel_std = data.std(dim=3).cpu().numpy()
                
                volume_mean = data.mean(dim=(0, 1, 2)).cpu().numpy()
                volume_std = data.mean(dim=(0, 1, 2)).cpu().numpy()
                
                run_mean = data.mean().cpu().item()
                run_std = data.std().cpu().item()
                
                require_dataset(run, 'voxel_mean', voxel_mean)
                require_dataset(run, 'voxel_std', voxel_std)
                require_dataset(run, 'volume_mean', volume_mean)
                require_dataset(run, 'volume_std', volume_std)
                
                run.attrs['run_mean'] = run_mean
                run.attrs['run_std'] = run_std
                
                B = data[..., None]
                
                A = torch.zeros_like(data)
                A[:, :, :, torch.arange(T)] = torch.arange(T).float().cuda()
                A = torch.stack([A, torch.ones_like(A)], dim=-1)
                
                X, _, _, _ = torch.linalg.lstsq(A, B)
                
                std = (A @ X - B).std(dim=3).squeeze().cpu().numpy()
                X = X.cpu().numpy()
                
                require_dataset(run, 'voxel_linear_trend', X)
                require_dataset(run, 'voxel_linear_trend_std', std)
                
                T = volume_mean.shape[0]
                B = torch.from_numpy(volume_mean[:, None])
                A = torch.arange(T).float()
                A = torch.stack([A, torch.ones_like(A)], dim=-1)
                
                X, _, _, _ = torch.linalg.lstsq(A, B)
                
                std = (A @ X - B).std()
                X = X.cpu().numpy()
                
                require_dataset(run, 'volume_linear_trend', X)
                require_dataset(run, 'volume_linear_trend_std', np.array([std]))
                
                #break
            #break
        #break

sub-01
imagery
<HDF5 group "/sub-01/imagery/runs/run_000" (1 members)>
<HDF5 group "/sub-01/imagery/runs/run_001" (1 members)>
<HDF5 group "/sub-01/imagery/runs/run_002" (1 members)>
<HDF5 group "/sub-01/imagery/runs/run_003" (1 members)>
<HDF5 group "/sub-01/imagery/runs/run_004" (1 members)>
<HDF5 group "/sub-01/imagery/runs/run_005" (1 members)>
<HDF5 group "/sub-01/imagery/runs/run_006" (1 members)>
<HDF5 group "/sub-01/imagery/runs/run_007" (1 members)>
<HDF5 group "/sub-01/imagery/runs/run_008" (1 members)>
<HDF5 group "/sub-01/imagery/runs/run_009" (1 members)>
<HDF5 group "/sub-01/imagery/runs/run_010" (1 members)>
<HDF5 group "/sub-01/imagery/runs/run_011" (1 members)>
<HDF5 group "/sub-01/imagery/runs/run_012" (1 members)>
<HDF5 group "/sub-01/imagery/runs/run_013" (1 members)>
<HDF5 group "/sub-01/imagery/runs/run_014" (1 members)>
<HDF5 group "/sub-01/imagery/runs/run_015" (1 members)>
<HDF5 group "/sub-01/imagery/runs/run_016" (1 members)>
<HDF5 group "/sub-01/imagery/runs

In [ ]:
with h5py.File('X:\\Datasets\\Deep-Image-Reconstruction\\derivatives\\kamitani2019.hdf5', 'r') as f:
    run = f['sub-01/natural_training/runs/run_001']
    data = run['data'][:]
    W, H, D, T = data.shape
    
    data = torch.from_numpy(data).cuda()
                
    voxel_mean = data.mean(dim=3).cpu().numpy()
    voxel_std = data.std(dim=3).cpu().numpy()

    volume_mean = data.mean(dim=(0, 1, 2)).cpu().numpy()
    volume_std = data.mean(dim=(0, 1, 2)).cpu().numpy()

    run_mean = data.mean().cpu().item()
    run_std = data.std().cpu().item()
    
    B = data[..., None]
                
    A = torch.zeros_like(data)
    A[:, :, :, torch.arange(T)] = torch.arange(T).float().cuda()
    A = torch.stack([A, torch.ones_like(A)], dim=-1)

    X, _, _, _ = torch.linalg.lstsq(A, B)

    std = (A @ X - B).std(dim=3).squeeze().cpu().numpy()
    X = X.cpu().numpy()

    T = volume_mean.shape[0]
    B = torch.from_numpy(volume_mean[:, None])
    A = torch.arange(T).float()
    A = torch.stack([A, torch.ones_like(A)], dim=-1)

    X, _, _, _ = torch.linalg.lstsq(A, B)

    std = (A @ X - B).std()
    print(np.array([std]))
    X = X.cpu().numpy()